In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.7/358.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 138.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 24.1 MB/s eta 0:00:00
  Attempting uninstall: torchao
    Found existing installation: torchao 0.10.0
    Uninstalling torchao-0.10.0:
     

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel

# Define model and tokenizer parameters
model_name = "unsloth/Llama-3.2-3B-Instruct"
max_seq_length = 1024           # Maximum sequence length for training/inference
dtype = None                     # Let the library auto-detect optimal data type

# Load the pre-trained LLaMA model and tokenizer with 4-bit quantization
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
from datasets import load_dataset

# Load the UC Irvine SMS Spam dataset
dataset = load_dataset("ucirvine/sms_spam")

# Inspect the column names to verify structure
print(dataset["train"].column_names)  # Expecting 'sms' for message, 'label' for class

# Format each example into the required prompt format for fine-tuning
def format_prompt(example):
    label_str = "HAM" if example['label'] == 0 else "SPAM"
    return f"### Text: {example['sms']}\n### Label: {label_str}<|endoftext|>"

# Apply formatting to all dataset splits and remove original columns
dataset = dataset.map(
    lambda x: {"text": format_prompt(x)},
    remove_columns=dataset["train"].column_names
)


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

['sms', 'label']


Map:   0%|          | 0/5574 [00:00<?, ? examples/s]

In [ ]:
# Split the original training dataset into training and validation sets
train_test = dataset["train"].train_test_split(test_size=0.2, seed=42)

# Assign splits to separate variables for clarity
train_dataset = train_test["train"]   # 80% for training
val_dataset = train_test["test"]      # 20% for validation


In [ ]:
# Apply LoRA adapters to the base model for parameter-efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # LoRA rank - controls adapter capacity
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Layers to apply LoRA
    lora_alpha=64,  # Scaling factor for LoRA updates
    lora_dropout=0.05,  # Dropout applied within LoRA layers for regularization
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.4 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
# Initialize SFTTrainer for supervised fine-tuning with LoRA-adapted model
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,  # Dataset used for training the model
    eval_dataset=val_dataset,     # Dataset used for evaluation during training
    dataset_text_field="text",    # Field in the dataset containing the text input
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of samples per GPU per step
        gradient_accumulation_steps=4,  # Accumulate gradients for effective batch size
        warmup_steps=20,                # Linear warmup steps for learning rate scheduler
        num_train_epochs=3,             # Total number of training epochs
        learning_rate=2e-4,             # Learning rate for optimizer
        fp16=True,                      # Mixed-precision training for faster performance
        logging_steps=25,               # Log metrics every N steps
        optim="adamw_8bit",             # Optimizer with 8-bit precision for memory efficiency
        weight_decay=0.1,               # Weight decay for regularization
        lr_scheduler_type="linear",     # Linear scheduler for learning rate
        seed=3407,                      # Seed for reproducibility
        output_dir="outputs",           # Directory to save model checkpoints
        report_to="none",               # Disable automatic logging (e.g., WandB)
        save_strategy="no"              # No automatic saving of checkpoints during training
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/4459 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1115 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,459 | Num Epochs = 3 | Total steps = 1,674
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 18,350,080 of 3,231,099,904 (0.57% trained)


Step,Training Loss
25,3.703900
50,2.892800
75,2.712500
100,2.526600
125,2.493000
150,2.480100
175,2.349400
200,2.408100
225,2.362300
250,2.347900


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
import pandas as pd

# Prepare model for inference mode to optimize generation
FastLanguageModel.for_inference(model)

# Initialize lists to store evaluation results
TP, TN, FP, FN = [], [], [], []
all_results = []

# Iterate over validation dataset to generate predictions
for idx, row in val_dataset.to_pandas().iterrows():
    text_input = row['text']
    # Determine ground-truth label based on dataset content
    true_label = "HAM" if "HAM" in text_input else "SPAM"

    # Apply chat template and tokenize input for model
    messages = [{"role": "user", "content": text_input}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # Generate text predictions using the model
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=50,
        temperature=0.1,
        do_sample=True,
        top_p=0.9
    )

    # Decode only the generated portion of the output
    pred_output = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0].strip().upper()

    # Parse predicted label from generated text
    pred_label = "SPAM" if "SPAM" in pred_output else "HAM"

    # Categorize prediction as TP, TN, FP, or FN
    match = ""
    if true_label == "SPAM":
        if pred_label == "SPAM":
            TP.append(text_input)
            match = "TP"
        else:
            FN.append(text_input)
            match = "FN"
    else:
        if pred_label == "SPAM":
            FP.append(text_input)
            match = "FP"
        else:
            TN.append(text_input)
            match = "TN"

    # Store detailed result for analysis
    all_results.append([text_input, true_label, pred_label, match])

# Compute evaluation metrics
num_TP, num_TN, num_FP, num_FN = len(TP), len(TN), len(FP), len(FN)
accuracy = (num_TP + num_TN) / (num_TP + num_TN + num_FP + num_FN)
precision = num_TP / (num_TP + num_FP) if (num_TP + num_FP) > 0 else 0
recall = num_TP / (num_TP + num_FN) if (num_TP + num_FN) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Print metrics summary
print("Metrics:")
print(f"Accuracy: {accuracy:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}")

# Convert evaluation results into a structured DataFrame for exporting
results_df = pd.DataFrame(
    all_results,
    columns=[
        "text",         # Original input text
        "true_label",   # Ground-truth label (HAM / SPAM)
        "pred_label",   # Model predicted label (HAM / SPAM)
        "result_type"   # Classification outcome: TP, TN, FP, FN
    ]
)

# Save results to Excel for further analysis or reporting
results_df.to_excel("model_predictions_comparison.xlsx", index=False)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Metrics:
Accuracy: 0.931, Precision: 0.686, Recall: 0.893, F1: 0.776


In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
from google.colab import files
import os

# Locate the fine-tuned GGUF model file and download it to the local machine for use or sharing
gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)
